In [1]:
from pathlib import Path
import geopandas as gpd

In [2]:
fn = Path(r"c:\Users\jpeacock\OneDrive - DOI\Geothermal\GreatBasin\sks_splitting_walpole_2014_02.shp")

In [3]:
gdf = gpd.read_file(fn)

# Convert GeoPandas GeoDataFrame to VTK (ParaView)
This adds helpers to export your GeoDataFrame as a .vtp (VTK PolyData) file readable by ParaView using PyVista. It supports Point/MultiPoint, LineString/MultiLineString, and Polygon/MultiPolygon.

Notes:
- If your CRS is geographic (degrees), consider reprojecting to a projected CRS in meters for better visualization.
- Attributes are exported: point features -> point data; line/polygon features -> cell data.
- Holes in polygons are ignored (ParaView draws the outer shell).

In [22]:
import numpy as np
import pyvista as pv
from shapely.geometry import (
    Point,
    MultiPoint,
    LineString,
    MultiLineString,
    Polygon,
    MultiPolygon,
)


def _polydata_from_points(coords: np.ndarray) -> pv.PolyData:
    # coords: (N, 3) or (N, 2)
    if coords.shape[1] == 2:
        coords = np.c_[coords, np.zeros(len(coords))]
    return pv.PolyData(coords)


def _polydata_from_lines(lines: list[np.ndarray]) -> pv.PolyData:
    # lines: list of arrays with shape (Ni, 3) or (Ni, 2)
    pts = []
    connectivity = []
    offset = 0
    for seg in lines:
        if seg.shape[1] == 2:
            seg = np.c_[seg, np.zeros(len(seg))]
        n = len(seg)
        pts.append(seg)
        # VTK polyline format: [npoints, i0, i1, ..., i(n-1)]
        connectivity.append(np.r_[n, np.arange(offset, offset + n)])
        offset += n
    if not pts:
        return pv.PolyData()
    pts = np.vstack(pts)
    lines_vtk = np.concatenate(connectivity).astype(np.int64)
    poly = pv.PolyData(pts)
    poly.lines = lines_vtk
    return poly


def _polydata_from_polygons(rings: list[np.ndarray]) -> pv.PolyData:
    # rings: list of exterior rings (holes ignored). Each ring shape (Ni, 3) or (Ni, 2)
    pts = []
    polys = []
    offset = 0
    for ring in rings:
        if ring.shape[1] == 2:
            ring = np.c_[ring, np.zeros(len(ring))]
        n = len(ring)
        pts.append(ring)
        polys.append(np.r_[n, np.arange(offset, offset + n)])
        offset += n
    if not pts:
        return pv.PolyData()
    pts = np.vstack(pts)
    polys_vtk = np.concatenate(polys).astype(np.int64)
    poly = pv.PolyData(pts)
    poly.polys = polys_vtk
    return poly


def geodf_to_pyvista(
    gdf: "gpd.GeoDataFrame",
    prefer_cell_data: bool = True,
    dscale=1000,
    center_point=(0, 0),
) -> pv.PolyData:
    """Convert a GeoDataFrame into a PyVista PolyData.

    - Points/MultiPoints -> point data
    - Lines/Polygons and their multi-variants -> cell data
    """
    if gdf.empty:
        return pv.PolyData()

    # Ensure we have geometry
    if "geometry" not in gdf.columns:
        raise ValueError("GeoDataFrame has no geometry column")

    # Ensure 2D->3D
    def to_xyz(geom):
        if hasattr(geom, "has_z") and geom.has_z:
            # already has z
            pass
        return geom

    geoms = gdf.geometry.apply(to_xyz)

    # Split by geom type
    points_coords = []
    line_parts = []
    poly_rings = []
    point_indices = []
    line_indices = []
    poly_indices = []

    print(center_point, dscale)
    for idx, geom in geoms.items():
        if isinstance(geom, Point):
            points_coords.append(
                [
                    (geom.y - center_point[1]) / dscale,
                    (geom.x - center_point[0]) / dscale,
                    getattr(geom, "z", 0.0),
                ]
            )
            point_indices.append(idx)
        elif isinstance(geom, MultiPoint):
            for p in geom.geoms:
                points_coords.append(
                    [
                        (p.y - center_point[1]) / dscale,
                        (p.x - center_point[0]) / dscale,
                        getattr(p, "z", 0.0),
                    ]
                )
                point_indices.append(idx)
        elif isinstance(geom, LineString):
            arr = np.asarray(
                [
                    (
                        (xy[1] - center_point[1]) / dscale,
                        (xy[0] - center_point[0]) / dscale,
                    )
                    for xy in geom.coords
                ]
            )
            line_parts.append(arr)
            line_indices.append(idx)
        elif isinstance(geom, MultiLineString):
            for ls in geom.geoms:
                arr = np.asarray(
                    [
                        (
                            (xy[1] - center_point[1]) / dscale,
                            (xy[0] - center_point[0]) / dscale,
                        )
                        for xy in ls.coords
                    ]
                )
                line_parts.append(arr)
                line_indices.append(idx)
        elif isinstance(geom, Polygon):
            arr = np.asarray(
                [
                    (
                        (xy[1] - center_point[1]) / dscale,
                        (xy[0] - center_point[0]) / dscale,
                    )
                    for xy in geom.exterior.coords
                ]
            )
            poly_rings.append(arr)
            poly_indices.append(idx)
        elif isinstance(geom, MultiPolygon):
            for pg in geom.geoms:
                arr = np.asarray(
                    [
                        (
                            (xy[1] - center_point[1]) / dscale,
                            (xy[0] - center_point[0]) / dscale,
                        )
                        for xy in pg.exterior.coords
                    ]
                )
                poly_rings.append(arr)
                poly_indices.append(idx)
        else:
            # unsupported geometries like GeometryCollection; skip
            pass

    # Build PolyData
    polydata_list = []
    mapping = {}

    if points_coords:
        pd_points = _polydata_from_points(np.asarray(points_coords, dtype=float))
        polydata_list.append(pd_points)
        mapping["points"] = (pd_points, point_indices)

    if line_parts:
        pd_lines = _polydata_from_lines(
            [np.asarray(p, dtype=float) for p in line_parts]
        )
        polydata_list.append(pd_lines)
        mapping["lines"] = (pd_lines, line_indices)

    if poly_rings:
        pd_polys = _polydata_from_polygons(
            [np.asarray(r, dtype=float) for r in poly_rings]
        )
        polydata_list.append(pd_polys)
        mapping["polys"] = (pd_polys, poly_indices)

    if not polydata_list:
        return pv.PolyData()

    mesh = polydata_list[0]
    for extra in polydata_list[1:]:
        mesh = mesh.merge(extra, main_has_priority=True)

    # Attach attributes
    attrs = gdf.drop(columns=["geometry"], errors="ignore")
    # Convert non-numeric columns to strings to avoid VTK issues
    safe_attrs = {}
    for col in attrs.columns:
        values = attrs[col]
        if values.dtype.kind in "biufc":
            safe_attrs[col] = np.asarray(values)
        else:
            safe_attrs[col] = values.astype(str).to_numpy()

    # Assign attributes
    if "points" in mapping and prefer_cell_data is False:
        for name, data in safe_attrs.items():
            # Broadcast to number of points; if multiple source features, this is ambiguous.
            if len(data) == len(gdf):
                mesh.point_data[name] = np.repeat(data[0], mesh.n_points)
            else:
                mesh.point_data[name] = data
    else:
        # put attributes on cells if we have lines/polys, otherwise on points
        target = (
            "cell_data" if ("lines" in mapping or "polys" in mapping) else "point_data"
        )
        for name, data in safe_attrs.items():
            arr = np.asarray(data)
            if target == "cell_data":
                n_cells = mesh.n_cells
                if len(arr) == len(gdf):
                    # Map per-feature attribute to each created VTK cell. We do a simple broadcast here.
                    mesh.cell_data[name] = np.resize(arr, n_cells)
                else:
                    mesh.cell_data[name] = np.resize(arr, n_cells)
            else:
                mesh.point_data[name] = np.resize(arr, mesh.n_points)

    return mesh


def save_geodf_as_vtk(
    gdf: "gpd.GeoDataFrame",
    out_path: str | Path,
    file_format: str = "vtp",
    center_point=(0, 0),
) -> Path:
    """Save a GeoDataFrame to a VTK file using PyVista.

    file_format in {'vtp', 'vtk'}; vtp recommended for PolyData.
    """
    mesh = geodf_to_pyvista(gdf, center_point=center_point)
    out_path = Path(out_path)
    if out_path.suffix.lower() not in (".vtp", ".vtk"):
        out_path = out_path.with_suffix(".vtp" if file_format == "vtp" else ".vtk")
    mesh.save(out_path)
    return out_path

In [28]:
# Example usage:
gdf_proj = gdf.to_crs(32611)  # project to UTM zone as needed
out_file = save_geodf_as_vtk(
    gdf_proj,
    fn.parent.joinpath("sks_splitting_gb_relative.vtp").as_posix(),
    center_point=(509041.3073876044 - 170000, 4189792.056445485+80000),
)
out_file

(339041.3073876044, 4269792.056445485) 1000


WindowsPath('c:/Users/jpeacock/OneDrive - DOI/Geothermal/GreatBasin/sks_splitting_gb_relative.vtp')

In [ ]:
gdf_proj.iloc[0].geometry.xy

(array('d', [508437.71545728715, 470968.90472829266]),
 array('d', [3643411.223970582, 3634819.935114808]))

In [8]:
model_center = (37.855540, -116.897222)

In [9]:
import pyproj

In [10]:
transformer = pyproj.Transformer.from_crs(4326, 32611)

In [14]:
transformer.transform(model_center[0], model_center[1])

(509041.3073876044, 4189792.056445485)